In [1]:
import numpy as np
import pandas as pd
import os
import json
from zipfile import ZipFile
import warnings
warnings.filterwarnings('ignore')

In [3]:
kaggle_credentials = json.load(open("kaggle.json"))

In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 43% 11.0M/25.7M [00:00<00:00, 112MB/s]
100% 25.7M/25.7M [00:00<00:00, 167MB/s]


In [6]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [7]:
temp_df = pd.read_csv('/content/IMDB Dataset.csv')

In [8]:
df = temp_df.iloc[:10000]

In [10]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
df['review'][1]

'wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen "has got polari" voices pat too! truly see seamless editing guided references williams\' diary entries, well worth watching terrificly written performed piece. masterful production one great master\'s comedy life. realism really comes home little things: fantasy guard which, rather use traditional \'dream\' techniques remains solid disappears. plays knowledge senses, particularly scenes concerning orton halliwell sets (particularly flat halliwell\'s murals decorating every surface) terribly well done.'

In [19]:
df.duplicated().sum()

0

In [20]:
df.drop_duplicates(inplace=True)

In [21]:
df.duplicated().sum()

0

In [22]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [23]:
df['review'] = df['review'].apply(remove_tags)

In [24]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [25]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [26]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [28]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [29]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [30]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [31]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [32]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

**BAG OF WORDS**

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [35]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [36]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_bow,y_train)

GaussianNB()

In [37]:
y_pred = gnb.predict(X_test_bow)
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6324486730095142

In [38]:
confusion_matrix(y_test,y_pred)

array([[717, 235],
       [499, 546]])

In [39]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

rfc.fit(X_train_bow,y_train)
y_pred = rfc.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8442663995993991

In [40]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.829243865798698

**N-GRAMS**

In [41]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8362543815723585

**TF-IDF**

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [43]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [44]:
rfc = RandomForestClassifier()

rfc.fit(X_train_tfidf,y_train)
y_pred = rfc.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

0.8452679018527791

**WORD2VEC**

In [45]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [47]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [48]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [49]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [50]:
model.build_vocab(story)

In [51]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875568, 6212140)

In [52]:
len(model.wv.index_to_key)

31845

In [53]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [54]:
document_vector(df['review'].values[0])

array([-0.12156092,  0.5094171 ,  0.22557576,  0.21476473, -0.10702705,
       -0.5967107 ,  0.20183371,  0.8857699 , -0.34162125, -0.34024173,
       -0.20169617, -0.4673596 ,  0.16040309,  0.05725668,  0.21097967,
       -0.1406007 ,  0.08072367, -0.38651457, -0.01744811, -0.6200913 ,
        0.09720565,  0.281707  ,  0.07213773, -0.25429592, -0.2758417 ,
       -0.0335764 , -0.2897604 ,  0.06534013, -0.310397  ,  0.06585641,
        0.31398055,  0.01596792,  0.1950368 , -0.26261473, -0.1388199 ,
        0.41680932,  0.15647812, -0.3324437 , -0.2717837 , -0.810522  ,
        0.12168279, -0.1721132 , -0.01313432, -0.1508631 ,  0.46490884,
       -0.07551385, -0.23597074, -0.07464827,  0.08157677,  0.3729003 ,
        0.04954966, -0.40060082, -0.3999422 , -0.15154865, -0.1415258 ,
        0.22104204,  0.17885262,  0.02394263, -0.31947547,  0.07480466,
       -0.00679316,  0.09244204,  0.05840402, -0.04513875, -0.38780227,
        0.2840251 ,  0.00194764,  0.1588318 , -0.35857183,  0.36

In [55]:
from tqdm import tqdm

X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [08:55<00:00, 18.65it/s]


In [56]:
X = np.array(X)

In [57]:
X[1]

array([-0.13043936,  0.5171091 ,  0.08232842, -0.00269278, -0.04261344,
       -0.45293117, -0.03275938,  0.7479607 , -0.22931774, -0.22708532,
       -0.5099745 , -0.45570266, -0.13415857,  0.2845621 ,  0.30908087,
       -0.13570891,  0.1722971 , -0.59789574,  0.01571713, -1.0377686 ,
        0.37554452,  0.26484147,  0.31601956,  0.02881859, -0.07371591,
        0.18575281, -0.22777504, -0.3091322 , -0.3725221 ,  0.02388931,
        0.6649482 ,  0.10285801,  0.27119133, -0.35038352, -0.08264343,
        0.54143447,  0.10582161, -0.074392  , -0.3489185 , -0.79709667,
        0.171219  , -0.42762733, -0.22613409,  0.12524152,  0.34053072,
       -0.31281552, -0.3078318 ,  0.03817486,  0.16779424,  0.32402062,
        0.26293978, -0.42592612, -0.4525184 ,  0.06884699, -0.33684513,
        0.1211352 ,  0.08341888, -0.26767853, -0.5077226 , -0.01723847,
       -0.16960639,  0.084697  , -0.1901646 ,  0.1399051 , -0.39545292,
        0.42702907,  0.10822286,  0.19169241, -0.29239088,  0.38

In [58]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [59]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [60]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)
accuracy_score(y_test,y_pred)

0.7681522283425137